# L03: MCP Tool Implementation

**Week 3 - Tool Use and Function Calling**

## Learning Objectives
- Understand the Model Context Protocol (MCP) architecture
- Implement MCP tools using the Python SDK
- Build a simple MCP server with custom tools
- Test tool invocation and response handling

## 1. Setup and Dependencies

MCP is Anthropic's open protocol for connecting LLMs to external tools.

In [ ]:
# Install MCP SDK (if not already installed)
# pip install mcp

import json
from typing import Any, Callable
from dataclasses import dataclass, field

print("Dependencies loaded successfully")

## 2. Understanding MCP Architecture

MCP follows a client-server architecture:
- **Host Application**: Contains the LLM and MCP client
- **MCP Client**: Manages connections to MCP servers
- **MCP Server**: Exposes tools, resources, and prompts

Communication uses JSON-RPC 2.0 over stdio (local) or HTTP (remote).

## 3. Simulating MCP Tools

We'll build a simplified MCP-style tool system to understand the concepts.

In [ ]:
@dataclass
class ToolParameter:
    """Represents a tool parameter with type and description."""
    name: str
    param_type: str
    description: str
    required: bool = True

@dataclass
class Tool:
    """MCP-style tool definition."""
    name: str
    description: str
    parameters: list = field(default_factory=list)
    handler: Callable = None
    
    def to_schema(self) -> dict:
        """Convert to JSON Schema format."""
        properties = {}
        required = []
        
        for param in self.parameters:
            properties[param.name] = {
                "type": param.param_type,
                "description": param.description
            }
            if param.required:
                required.append(param.name)
        
        return {
            "name": self.name,
            "description": self.description,
            "input_schema": {
                "type": "object",
                "properties": properties,
                "required": required
            }
        }

print("Tool classes defined")

In [ ]:
class MCPServer:
    """Simplified MCP server implementation."""
    
    def __init__(self, name: str):
        self.name = name
        self.tools: dict[str, Tool] = {}
    
    def register_tool(self, tool: Tool):
        """Register a tool with the server."""
        self.tools[tool.name] = tool
        print(f"Registered tool: {tool.name}")
    
    def list_tools(self) -> list[dict]:
        """List all available tools (MCP tools/list)."""
        return [tool.to_schema() for tool in self.tools.values()]
    
    def call_tool(self, name: str, arguments: dict) -> dict:
        """Execute a tool (MCP tools/call)."""
        if name not in self.tools:
            return {"error": f"Tool '{name}' not found"}
        
        tool = self.tools[name]
        if tool.handler is None:
            return {"error": f"Tool '{name}' has no handler"}
        
        try:
            result = tool.handler(**arguments)
            return {"content": [{"type": "text", "text": str(result)}]}
        except Exception as e:
            return {"error": str(e)}

print("MCPServer class defined")

## 4. Implementing Custom Tools

In [ ]:
# Tool handlers
def calculate_handler(expression: str) -> str:
    """Safely evaluate a mathematical expression."""
    # Only allow safe math operations
    allowed_chars = set('0123456789+-*/.() ')
    if not all(c in allowed_chars for c in expression):
        return "Error: Invalid characters in expression"
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {e}"

def get_weather_handler(location: str) -> str:
    """Simulated weather lookup."""
    # In production, this would call a real weather API
    weather_data = {
        "new york": {"temp": "45F", "condition": "Cloudy"},
        "london": {"temp": "50F", "condition": "Rainy"},
        "tokyo": {"temp": "55F", "condition": "Sunny"},
    }
    loc_lower = location.lower()
    if loc_lower in weather_data:
        data = weather_data[loc_lower]
        return f"Weather in {location}: {data['temp']}, {data['condition']}"
    return f"Weather data not available for {location}"

def search_web_handler(query: str, num_results: int = 3) -> str:
    """Simulated web search."""
    # In production, this would call a real search API
    return f"Search results for '{query}': [Simulated {num_results} results]"

print("Tool handlers defined")

In [ ]:
# Create MCP server and register tools
server = MCPServer("demo-server")

# Calculator tool
calc_tool = Tool(
    name="calculate",
    description="Evaluate a mathematical expression. Supports +, -, *, /, parentheses.",
    parameters=[
        ToolParameter("expression", "string", "The mathematical expression to evaluate")
    ],
    handler=calculate_handler
)
server.register_tool(calc_tool)

# Weather tool
weather_tool = Tool(
    name="get_weather",
    description="Get current weather for a location. Returns temperature and conditions.",
    parameters=[
        ToolParameter("location", "string", "City name (e.g., 'New York', 'London')")
    ],
    handler=get_weather_handler
)
server.register_tool(weather_tool)

# Search tool
search_tool = Tool(
    name="search_web",
    description="Search the web for current information. Use for recent events or facts.",
    parameters=[
        ToolParameter("query", "string", "Search query"),
        ToolParameter("num_results", "integer", "Number of results", required=False)
    ],
    handler=search_web_handler
)
server.register_tool(search_tool)

## 5. Testing Tool Invocation

In [ ]:
# List available tools
print("Available Tools:")
print("=" * 50)
for tool_schema in server.list_tools():
    print(json.dumps(tool_schema, indent=2))
    print("-" * 50)

In [ ]:
# Test tool calls
print("Tool Call Tests:")
print("=" * 50)

# Test calculator
result = server.call_tool("calculate", {"expression": "(10 + 5) * 2"})
print(f"calculate('(10 + 5) * 2'): {result}")

# Test weather
result = server.call_tool("get_weather", {"location": "Tokyo"})
print(f"get_weather('Tokyo'): {result}")

# Test search
result = server.call_tool("search_web", {"query": "MCP protocol", "num_results": 5})
print(f"search_web('MCP protocol'): {result}")

# Test error handling
result = server.call_tool("unknown_tool", {})
print(f"unknown_tool(): {result}")

## 6. Simulating LLM Tool Use Flow

In production, the LLM would:
1. Receive user query
2. Decide which tool(s) to call
3. Generate tool call with arguments
4. Receive tool result
5. Generate final response

In [ ]:
def simulate_agent_flow(user_query: str, tool_decision: dict):
    """Simulate the full agent tool-use flow."""
    print(f"User Query: {user_query}")
    print("-" * 50)
    
    # Step 1: LLM decides to use a tool
    tool_name = tool_decision["tool"]
    tool_args = tool_decision["arguments"]
    print(f"LLM Decision: Call {tool_name} with {tool_args}")
    
    # Step 2: Execute the tool
    result = server.call_tool(tool_name, tool_args)
    print(f"Tool Result: {result}")
    
    # Step 3: LLM generates final response
    if "content" in result:
        print(f"Final Response: Based on the tool result, {result['content'][0]['text']}")
    else:
        print(f"Final Response: I encountered an error: {result['error']}")
    print("=" * 50)

# Simulate queries
simulate_agent_flow(
    "What's 15 times 24?",
    {"tool": "calculate", "arguments": {"expression": "15 * 24"}}
)

simulate_agent_flow(
    "What's the weather like in London?",
    {"tool": "get_weather", "arguments": {"location": "London"}}
)

## 7. Key Takeaways

1. **Tool Schema**: Tools are defined with name, description, and JSON Schema parameters
2. **MCP Server**: Exposes tools via `list_tools` and `call_tool` methods
3. **Error Handling**: Always validate inputs and handle exceptions
4. **Security**: Sanitize inputs (see calculator example)
5. **Descriptions**: Clear descriptions help LLMs choose the right tool

## Next Steps
- Explore the official MCP SDK at `github.com/modelcontextprotocol`
- Build a real MCP server with stdio transport
- Connect your MCP server to Claude Desktop